The sparksession instance is the way Spark executes user-defined manipulation across the cluster

In [ ]:
# start a spark session


## DBFS (Databrick File System)

More detail can be found [here](https://docs.databricks.com/data/databricks-file-system.html)

Type dbutils.fs.help() for help.

In [ ]:
dbutils.fs.help()

In [ ]:
# List current directory
dbutils.fs.ls('/')

Notebooks support a shorthand—%fs magic commands—for accessing the dbutils filesystem module. Most dbutils.fs commands are available using %fs magic commands

In [ ]:
# List current directory


#### TODO #1: Create a new directory called `data1050` and show where it locates

## An End to End example
We will use spark to analyze some flight data from the United States Bureau of Transporation statistics

Data in this example is located at this directory: /databricks-datasets/definitive-guide/data

You can also download the data and all notebooks of the text on Gitbub https://github.com/databricks/Spark-the-Definitive-guide

##### Read data into Spark DataFrame

In [ ]:
# read 2015 summary data
path= "/databricks-datasets/definitive-guide"
flightData2015 = spark\
  .read\
  .option("inferSchema", "false")\
  .option("header", "false")\
  .csv(path+"/data/flight-data/csv/2015-summary.csv")

In [ ]:
df = spark.read.csv(path+"/data/flight-data/csv/2015-summary.csv", header=True, inferSchema=True)

In [ ]:
# take a look at our schema


In [ ]:
df

##### Read data into SQL Table

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS flightData2015
USING CSV
OPTIONS (path '/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv', header 'true', inferSchema 'true');

In [ ]:
%sql
SELECT * FROM flightData2015 LIMIT 10;

In [ ]:
# covert the DataFrame to a table so that we can use sql to query the table
df.createOrReplaceTempView("flight_data_2015")

##### SQL Table vs. Spark DataFrame

In [ ]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY count(1) DESC
""")

sqlWay.show()

In [ ]:
dataFrameWay = df\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()\
  .orderBy("count", ascending=False)

dataFrameWay.show()

In [ ]:
sqlWay.explain()
dataFrameWay.explain()

##### PySpark Functions

In [ ]:
from pyspark.sql.functions import *

#### TODO #2: 